# Song Recommender Data Science Project

### Imports for this Project

In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Loading in the songs with tracks.csv and doing slight data cleaning

In [50]:
# Loading in the songs csv file
songs = pd.read_csv("tracks.csv")

# Renaming specific columns
renamed = songs.rename(columns={"id":"song id", "name":"song name", "artists": "artist", "id_artists": "artist id", "release_date": "release date"})

# Checking the shape of the dataset
renamed.shape

(586672, 20)

In [35]:
# Checking to see what the renamed dataframe looks like:
renamed.head()

,song id,song name,popularity,duration_ms,explicit,artist,artist id,release date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


# What each function below does:
scale_playlist(dataframe):
- It identifies columns that represent continuous numerical values (e.g., 'danceability', 'energy', etc.).
- These columns are normalized using the StandardScaler, which standardizes the data by removing the mean and scaling to unit variance.
- The normalized values are then multiplied by 0.2 to scale them down further.
- The 'popularity' column is also scaled using the same method but multiplied by 0.1.
- After scaling, the function merges the scaled features with other relevant song metadata ('song id', 'artist id', etc.) and returns the resulting dataframe.

generate_playlist_feature(complete_feature_set, standardized_playlist_df)
- Sums up all the feature values of the songs in the provided playlist (standardized_playlist_df) along the columns (for all features).
- Drops non-feature columns like 'song id', 'artist id', 'song name', and 'artist'.
- The remaining features are returned as a vector that represents the overall "signature" or average profile of the playlist.

cosine_sim1(pl1, pl2)
- Computes the cosine similarity between two playlists (pl1, pl2). Cosine similarity measures how similar two vectors are in terms of their direction, which is useful for comparing feature vectors.
- Also computes the dot product between the vectors and divides it by the product of their magnitudes (Euclidean norms).

songs_table_with_similarity(a_songs_dataframe, a_vector)
- Adds a column of cosine similarity values to a dataframe of songs (a_songs_dataframe), comparing each song's feature vector with a given playlist vector (a_vector):
- Removes the last four columns (assumed to be non-feature columns like 'song id', 'artist', etc.) to focus only on the numerical feature columns.
- Computes cosine similarity between each song's feature vector and the input playlist vector (a_vector).
- The similarity values are stored in a new column ('cosine similarities'), and the songs are sorted by their similarity to the playlist vector.

generate_complete_playlist(entire_dataframe, existing_playlist_dataframe, new_playlist_size)
- Generates a new playlist by finding songs from the entire dataset that are most similar to an existing playlist:
- Scales both the entire song dataset and the existing playlist using scale_playlist.
- Then it generates a feature vector for the existing playlist using generate_playlist_feature.
Then, it calculates cosine similarity between each song in the dataset and the feature vector of the existing playlist using songs_table_with_similarity.
A list of song IDs from the existing playlist is used to filter out duplicates, ensuring that no song already in the existing playlist is selected.
Finally, the function returns a list of the most similar songs that aren't already in the existing playlist, limited to the size of the new playlist (new_playlist_size).

In [76]:
def scale_playlist(df):
    # These are the columns that have float values
    float_cols = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    # Resetting the index for the float columns and prepare the data for scaling
    floats = df[float_cols].reset_index(drop = True)
    
    scaler = StandardScaler()
    # Using Standard Scaler to scale these float columns that we just specified
    # We are scaling the float columns by multiplying by 0.2 as th efinal scaling
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Popularity will now be a different column so we can scale it separately
    pop = df[["popularity"]].reset_index(drop = True)
    scaler = StandardScaler()
    # Scaling the popularity column by multiplying by 0.1
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.1

    # Adding the relevant metadata back to our new scaled dataframe
    final = pd.concat([floats_scaled, pop_scaled], axis = 1)
    final['song id']=df['song id'].values
    final['artist id']=df['artist id'].values
    final['song name']=df['song name'].values
    final['artist']=df['artist'].values
    return final


# Generating a single feature vector that represents the combined features of the playlist
def generate_playlist_feature(complete_feature_set, standardized_playlist_df):
    # Sum up all features across the playlist and drop non-feature columns
    return standardized_playlist_df.sum(axis = 0).drop('song id').drop('artist id').drop('song name').drop('artist')


# Calculating cosine similarity between two feature vectors
def cosine_sim1(pl1, pl2):
    # Compute the cosine similarity by taking the dot product of two vectors
    cosine_sim = np.dot(np.array(pl1), np.array(pl2))/(np.linalg.norm(np.array(pl1)) * np.linalg.norm(np.array(pl2)))
    return cosine_sim


# Creating a table of cosine similarity between each song and a given playlist vector
def songs_table_with_similarity(a_songs_dataframe, a_vector):
    '''
    Input: 
    a_songs_dataframe: spotify dataframe
    a_vector: vector of a playlist we want to get similar songs for

    Output:
    a new table with an added column with a cosine similarity value () for each song in the dataframe compared to a_vector
    '''
    
    # creates a list of lists of all of the rows in the dataframe
    # it also removes the last 4 columns (assumed to be non-feature columns like song metadata) from the dataframe
    dataframe_without_strings = a_songs_dataframe.iloc[:, :-4]

    # Convert the numerical part of the dataframe into a list of lists
    convert_list = dataframe_without_strings.values.tolist()

    # find the cosine similarity between each row of the dataframe and the inputted vector
    # We are first initializing an empty list to store cosine similarity values before we loop through each row in the list
    cosine_sims = []
    for i in convert_list:
        one_cosine_sim = cosine_sim1(i, a_vector)
        cosine_sims.append(one_cosine_sim)
    
    # create a new column with the cosine similarities
    dataframe_without_strings['cosine similarities'] = cosine_sims

    # return a table with the songs with the highest cosine similarities on top
    return dataframe_without_strings.sort_values(['cosine similarities'], ascending=False).merge(a_songs_dataframe)


# Generating a complete playlist based on similarities and avoiding duplicate songs
def generate_complete_playlist(entire_dataframe, existing_playlist_dataframe, new_playlist_size):
    # Extract the song IDs from the existing playlist to avoid duplicates
    duplicate_song_id_list = existing_playlist_dataframe["song id"].values.tolist()

    # Scale the entire song dataframe and the existing playlist
    entire_dataframe_scaled = scale_playlist(entire_dataframe)
    existing_playlist_dataframe_scaled = scale_playlist(existing_playlist_dataframe)

    # Generate a feature vector representing the existing playlist                                             
    features_vector = generate_playlist_feature(entire_dataframe, existing_playlist_dataframe)
    print("The singular features vector representing all the songs in the existing playlist:")
    display(features_vector)
                                                        
    # Mark songs that are already in the existing playlist to avoid duplications, and filter out duplicate songs
    songs_with_duplicates = songs_table_with_similarity(entire_dataframe_scaled, features_vector)
    songs_with_duplicates['duplicate'] = songs_with_duplicates['song id'].str.contains('|'.join(duplicate_song_id_list)).head(new_playlist_size)
    songs_without_duplicates = songs_with_duplicates[songs_with_duplicates['duplicate']==False]
    
    return songs_without_duplicates


In [77]:
renamed_scaled = scale_playlist(renamed)

# Generating playlists based on specific artists, i.e. the first playlist is a playlist of songs from just paramore
playlist_1 = renamed_scaled[renamed_scaled["artist"].str.contains('Paramore')]
playlist_2 = renamed_scaled[renamed_scaled["artist"].str.contains('John Mayer')]
playlist_3 = renamed_scaled[renamed_scaled["artist"].str.contains('Metallica')]

# Generating a playlist for a specific artist based on their artist id
playlist_4 = renamed_scaled[renamed_scaled["artist id"].str.contains('3GBPw9NK25X1Wt2OUvOwY3')]

# Generating additional playlists for various artists
playlist_5 = renamed_scaled[renamed_scaled["artist"].str.contains('JID')]
playlist_6 = renamed_scaled[renamed_scaled["artist"].str.contains('Kendrick Lamar')]
playlist_7 = renamed_scaled[renamed_scaled["artist"].str.contains('J. Cole')]
playlist_8 = renamed_scaled[renamed_scaled["artist"].str.contains('100 gecs')]
playlist_9 = renamed_scaled[renamed_scaled["artist"].str.contains('Clairo')]
playlist_10 = renamed_scaled[renamed_scaled["artist"].str.contains('Ed Sheeran')]
playlist_11 = renamed_scaled[renamed_scaled["artist"].str.contains('Taylor Swift')]

In [78]:
# Checksif a given artist exists in the dataframe
def does_artist_exist(df, artist_name):
    list_of_artists = df.groupby('artist', as_index=False).agg('sum')['artist']
    artist_count = list_of_artists.str.contains(artist_name).sum()
    if artist_count > 0:
        print("Yes, this DataFrame contains the artist: " + artist_name + ".")
    else:
        print("Sorry, this DataFrame does not contain the artist: " + artist_name + ".")

# Checks if a specific song by a given artist exists in the dataframe
def does_song_exist(df, song_name, artist_name):
    artist_df = df[df['artist'].str.contains(artist_name)]
    song_count = artist_df['song name'].str.contains(song_name).sum()
    if song_count > 0:
        artist_s_df = artist_df[artist_df['song name'].str.contains(song_name)]['artist'].iloc[0]
        song_id = artist_df[artist_df['song name'].str.contains(song_name)]['song id'].iloc[0]
        print("Yes, this DataFrame contains the song: " + song_name + " by: " + artist_s_df + ". The song id is: " + song_id)
    else :
        print("Sorry, this DataFrame does not contain the song: " + song_name + " by: " + artist_name + ".")

In [79]:
# Confirms that the artist, Paramore, exists in the dataset
does_artist_exist(renamed, "Paramore")

Yes, this DataFrame contains the artist: Paramore.


In [80]:
# Confirms that nonexistent artists do not exist
does_artist_exist(renamed, "Tkdsmowrsd")

Sorry, this DataFrame does not contain the artist: Tkdsmowrsd.


In [81]:
# Now, we will create a recommendation playlist based on the songs that are by Taylor Swift.
# First, we generate a playlist for 'Taylor Swift
taylor_playlist = renamed_scaled[renamed_scaled["artist"].str.contains('Taylor Swift')]
print("The original feature vectors for each song:")
display(taylor_playlist)
generate_complete_playlist(renamed, taylor_playlist, 10)

The original feature vectors for each song:


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,song id,artist id,song name,artist
79931,0.124509,0.092857,0.207260,-0.083454,-0.193135,-0.085024,0.120510,0.017625,-0.197129,0.187418,43H3q2NWT9LCuYS1B5IUvV,['06HL4z0CvFAxyc27GXpf02'],Our Song,['Taylor Swift']
80133,0.069121,-0.099265,0.128310,-0.090903,-0.092802,-0.085024,-0.103008,-0.204364,-0.124391,0.171088,2TF4UtYreqNbQ6Z9AccldU,['06HL4z0CvFAxyc27GXpf02'],Teardrops On My Guitar - Radio Single Remix,['Taylor Swift']
80310,0.075142,-0.081006,0.128271,-0.090791,-0.068148,-0.085024,-0.073712,-0.227649,-0.124330,0.165645,7snx7w5WZLHovS0xC45ZfP,['06HL4z0CvFAxyc27GXpf02'],Teardrops On My Guitar - Radio Single Remix,['Taylor Swift']
81277,0.064305,0.157956,0.245065,-0.082008,-0.182815,-0.085024,-0.148363,-0.191168,0.003488,0.274514,1D4PL9B8gOg78jiHg3FvBb,['06HL4z0CvFAxyc27GXpf02'],Love Story,['Taylor Swift']
81319,0.148590,0.161132,0.226909,-0.074448,-0.155868,-0.085012,-0.111688,-0.091041,0.077282,0.252740,1GEBsLDvJGw7kviySRI6GX,['06HL4z0CvFAxyc27GXpf02'],You Belong With Me,['Taylor Swift']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576101,0.048652,-0.019876,0.046846,-0.078006,0.182399,-0.084908,-0.098668,-0.032051,0.063850,0.236410,1XjHRolIXL2M1EEOUsGGR4,['06HL4z0CvFAxyc27GXpf02'],cowboy like me,['Taylor Swift']
576103,0.034203,0.124613,0.047592,-0.064220,0.117612,-0.085011,-0.108433,-0.035931,0.157580,0.236410,2QDyYdZyhlP2fp79KZX8Bi,['06HL4z0CvFAxyc27GXpf02'],mad woman,['Taylor Swift']
576106,-0.032021,-0.003998,-0.041653,-0.047988,0.211639,-0.084347,-0.078052,-0.202035,-0.071024,0.241853,3k7ne7VmH43ZPWxPdvPUgR,"['06HL4z0CvFAxyc27GXpf02', '2cCUtGK9sDU2EoElnk...",coney island (feat. The National),"['Taylor Swift', 'The National']"
576109,0.100427,-0.214380,-0.106102,-0.014746,0.268399,-0.085011,-0.132521,-0.165554,0.212196,0.230966,7MbT4I8qGntX4fMdqMQgke,['06HL4z0CvFAxyc27GXpf02'],peace,['Taylor Swift']


The singular features vector representing all the songs in the existing playlist:


danceability        11.274256
energy              26.646237
loudness            61.164162
speechiness        -24.744294
acousticness       -50.769857
instrumentalness   -32.792998
liveness           -27.850151
valence            -36.431198
tempo               13.480717
popularity          71.438387
dtype: object

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,cosine similarities,song id,artist id,song name,artist,duplicate
0,0.022162,0.119849,0.202701,-0.064887,-0.192562,-0.085024,-0.091073,-0.164002,0.049987,0.279957,0.989427,3ndCsZfq8OVwwkz9mmZvUz,['3Nrfpe0tUJi4K4DXYWgMUX'],Stay,['BTS'],False
1,0.010122,0.118262,0.203094,-0.081675,-0.164468,-0.085024,-0.080222,-0.149254,0.084055,0.252740,0.987797,0nTXElyiOEN60Z22Pyu9tt,['3b8QkneNDz4JHKKKlLgYZg'],Life Rolls On,['Florida Georgia Line'],False
2,0.024571,0.125407,0.233079,-0.065888,-0.190842,-0.085024,-0.130242,-0.088712,0.043537,0.285401,0.985815,0PDUDa38GO8lMxLCRc4lL1,['5ZsFI1h6hIdQRw2ti0hz81'],PILLOWTALK,['ZAYN'],False
3,0.077550,0.077773,0.237441,-0.061996,-0.201047,-0.085024,-0.082392,-0.111221,0.076267,0.225523,0.984286,38SKB7UfhL6Sd6Joxex5yK,['41MozSoPIsD1dJM0CLPjZF'],Don't Know What To Do,['BLACKPINK'],False
4,0.064305,0.112704,0.258898,-0.067778,-0.226675,-0.085024,-0.119284,-0.199706,0.091709,0.263627,0.983631,05mDaV9Vb3wrzjF6OPZnhq,['246dkjvS1zLTtiykXe5h60'],Saint-Tropez,['Post Malone'],False
5,0.077550,0.077773,0.237441,-0.061996,-0.201047,-0.085024,-0.082392,-0.111221,0.076267,0.214636,0.982590,4ghGrdxwqLA6ZoCiu0u4IR,['41MozSoPIsD1dJM0CLPjZF'],Don't Know What To Do,['BLACKPINK'],False
6,-0.001919,0.111911,0.227105,-0.083676,-0.254877,-0.085024,-0.116028,-0.129850,0.090970,0.252740,0.979937,6vC90OOjZR165Hw8CpsqEm,['0Z8XVUAOBPM4x12wKnFHEQ'],Pretty Heart,['Parker McCollum'],False
7,0.005305,0.118262,0.195746,-0.082564,-0.150135,-0.085024,-0.138706,-0.121312,0.084277,0.209192,0.979220,1a0PcmEJAOEp4ZT3YtSLkM,['4AK6F7OLvEQ5QYCBNiQWHq'],Through the Dark,['One Direction'],False
8,0.053468,0.057132,0.254614,-0.084899,-0.127775,-0.085021,-0.061777,-0.116655,0.063292,0.236410,0.979214,3UN6cIn3VIyg0z1LCuFSum,"['1Cs0zKBU1kc0i8ypK3B9ai', '23zg3TcAtWQy7J6upg...",Without You (feat. Usher),"['David Guetta', 'Usher']",False
9,-0.021184,0.091269,0.212958,-0.079674,-0.204716,-0.085024,-0.103008,-0.148478,0.098932,0.236410,0.978907,1ZWLWVqeEMWMKTlteS0yLH,['4AK6F7OLvEQ5QYCBNiQWHq'],Love You Goodbye,['One Direction'],False


In [87]:
# Only giving us the songs name and artist(s)
generate_complete_playlist(renamed, taylor_playlist, 10)[['song name','artist']]

The singular features vector representing all the songs in the existing playlist:


danceability        11.274256
energy              26.646237
loudness            61.164162
speechiness        -24.744294
acousticness       -50.769857
instrumentalness   -32.792998
liveness           -27.850151
valence            -36.431198
tempo               13.480717
popularity          71.438387
dtype: object

,song name,artist
0,Stay,['BTS']
1,Life Rolls On,['Florida Georgia Line']
2,PILLOWTALK,['ZAYN']
3,Don't Know What To Do,['BLACKPINK']
4,Saint-Tropez,['Post Malone']
5,Don't Know What To Do,['BLACKPINK']
6,Pretty Heart,['Parker McCollum']
7,Through the Dark,['One Direction']
8,Without You (feat. Usher),"['David Guetta', 'Usher']"
9,Love You Goodbye,['One Direction']


### Testing the function on one of my own Spotify playlists
#### This playlist contains a lot of rap and R&B

In [83]:
# Checking if 'Changes' by 2Pac is in our dataset
does_song_exist(renamed, "Changes", "2Pac")

Yes, this DataFrame contains the song: Changes by: ['2Pac', 'Talent']. The song id is: 1ofhfV90EnYhEr7Un2fWiv


In [92]:
# After manually checking to see if every song in my 24-song playist is present in the dataset, I create my_playlist as a dataframe with 24 rows
my_playlist_ids = ['0JXXNGljqupsJaZsgSbMZV', '1ofhfV90EnYhEr7Un2fWiv', '1aWV3uY3SIEZVbmv45oFWS', '0I3q5fE6wg7LIfHGngUTnV', '7KwZNVEaqikRSBSpyhXK2j', '2Q21T7gfWltYB592v39re5', '5mCPDVBb16L4XQwDdbRUpz', '6brl7bwOHmGFkNw3MBqssT','3BtuIIrQlkujKPuWF2B85z', '2aI21FnmY7TJVKeMaoQZ0t', '2qOm7ukLyHUXWyR4ZWLwxA', '1tkg4EHVoqnhR6iFEXb60y', '285pBltuF7vW8TeWk8hdRR', '62vpWI1CHwFy7tMIcSStl8', '2g8HN35AnVGIk7B8yMucww', '4AE7Lj39VnSZNOmGH2iZaq', '1VLtjHwRWOVJiE5Py7JxoQ', '6jdOi5U5LBzQrc4c1VT983', '2Ih217RCGAmyQR68Nn7Cqo', '6fbjdXZNtoAJbJZUeT87Ii', '5fQBa4wkmq28xpSLOQ202K', '7G3lxTsMfSx4yarMkfgnTC', '5VA4Ispp52EA1sOqzMz3Av', '0qtK3XwbuG153dmwB8iepL']
my_playlist = renamed_scaled[renamed_scaled['song id'].str.contains('|'.join(my_playlist_ids))]
my_playlist.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,song id,artist id,song name,artist
65440,0.282243,0.160338,0.191698,0.034617,-0.068721,-0.084945,0.084703,0.187610,-0.242634,0.279957,2qOm7ukLyHUXWyR4ZWLwxA,['3Mcii5XWf6E0lrY3Uky4cA'],It Was A Good Day,['Ice Cube']
65763,0.352079,0.198445,0.273123,0.011269,-0.148415,-0.085024,0.204058,0.192267,-0.117019,0.143871,2Q21T7gfWltYB592v39re5,['3Mcii5XWf6E0lrY3Uky4cA'],Check Yo Self,['Ice Cube']
66443,0.220835,-0.017494,-0.012455,-0.076894,-0.201907,-0.044779,-0.078052,0.037806,-0.171992,0.258184,2Ih217RCGAmyQR68Nn7Cqo,['3Mcii5XWf6E0lrY3Uky4cA'],You Know How We Do It - Remastered,['Ice Cube']
67440,0.260569,0.026170,0.116285,0.186930,-0.011388,-0.085024,-0.076967,0.171310,-0.228281,0.279957,2g8HN35AnVGIk7B8yMucww,['5me0Irg2ANcsgc93uaYrpb'],Big Poppa - 2005 Remaster,['The Notorious B.I.G.']
67488,0.297896,-0.066716,0.020359,-0.071891,-0.246857,-0.085023,-0.175055,0.189938,-0.171514,0.220079,1aWV3uY3SIEZVbmv45oFWS,['0TImkz4nPqjegtVSMZnMRq'],Creep,['TLC']


In [95]:
# Generating a 20-song long playlist based on my_playlist
generate_complete_playlist(renamed, my_playlist, 20)[['song name','artist']]

The singular features vector representing all the songs in the existing playlist:


danceability        4.915938
energy              1.231438
loudness            2.802793
speechiness         1.832398
acousticness       -3.390985
instrumentalness   -1.931738
liveness            -1.01846
valence             0.086918
tempo              -3.955886
popularity          6.207294
dtype: object

,song name,artist
0,Boys Like You,['Anna Clendening']
1,"Young, Wild & Free (feat. Bruno Mars)","['Snoop Dogg', 'Wiz Khalifa', 'Bruno Mars']"
2,Agua (with J Balvin),"['Tainy', 'J Balvin']"
3,wokeuplikethis*,"['Playboi Carti', 'Lil Uzi Vert']"
4,ROCKSTAR (feat. Roddy Ricch),"['DaBaby', 'Roddy Ricch']"
5,LA NOCHE DE ANOCHE,"['Bad Bunny', 'ROSALÍA']"
6,Rich Girl,['Lake Street Dive']
7,Heartless,['Kanye West']
8,Ain't It Different (feat. AJ Tracey & Stormzy),"['Headie One', 'AJ Tracey', 'Stormzy']"
9,Vete,['Bad Bunny']
